# Importing request

In [4]:
import requests
import json
from pprint import pprint
import pandas as pd
from config import api_key
from config import api_id

# API Call
using a for loop to loop through three years work of PM2.5 data

In [5]:
state_code = '06'
county = '075'

response_sample = []
start = ["20190101", "20180101","20170101"]
end = ["20191231","20181231","20171231"]


for index in range(len(start)):
    # for each_state_code in ga_county_codes:
    url = f"https://aqs.epa.gov/data/api/sampleData/byCounty?email={api_id}&key={api_key}&param=88101&bdate={start[index]}&edate={end[index]}&state={state_code}&county={county}"
    response_sample.append(requests.get(url).json())

# Pulling Data and putting in list

In [6]:
time = []
date = []
county = []
lat = []
lon = []
site = []
sample = []


for x in range(0,3):
    for y in range(0,8688):
        time.append(response_sample[x]['Data'][y]['time_local'])
        date.append(response_sample[x]['Data'][y]['date_local'])
        county.append(response_sample[x]['Data'][y]['county'])
        lat.append(response_sample[x]['Data'][y]['latitude'])
        lon.append(response_sample[x]['Data'][y]['longitude'])
        site.append(response_sample[x]['Data'][y]['site_number'])
        sample.append(response_sample[x]['Data'][y]['sample_measurement'])

        
columns = ['time','date','county','lat','lon','site','sample']
df_sample = pd.DataFrame(data = list(zip(time,date,county,lat,lon,site,sample)), columns = columns)
df_sample.head()

,time,date,county,lat,lon,site,sample
0,00:00,2019-01-01,San Francisco,37.765946,-122.399044,0005,5.0
1,01:00,2019-01-01,San Francisco,37.765946,-122.399044,0005,2.0
2,02:00,2019-01-01,San Francisco,37.765946,-122.399044,0005,2.0
3,03:00,2019-01-01,San Francisco,37.765946,-122.399044,0005,4.0
4,04:00,2019-01-01,San Francisco,37.765946,-122.399044,0005,2.0


checking number of distinct sites

In [78]:
df_sample.site.unique()

array(['0005'], dtype=object)

# Grouping Hourly Data by day & Merging
PM2.5 data is organized by hour, but we need it on a daily level so it can map in with the AQI levels. First we do a groupby to get daily levels, then we need to remerge with the original data to get the categorical data back in (county, site, lat, lon)

In [86]:
df_sample1 = df_sample[['date','sample']].groupby('date').mean().reset_index().sort_values('date',ascending = False)
df_sample1.head()

,date,sample
1089,2019-12-29,9.400000
1088,2019-12-28,9.208333
1087,2019-12-27,8.208333
1086,2019-12-26,7.043478
1085,2019-12-25,3.500000


In [90]:
# data is all coming from same lat/lon and county
df_sample2 = df_sample.drop(columns = ['time','sample','site']).drop_duplicates().sort_values('date',ascending = False)
df_sample2.head()

,date,county,lat,lon
8683,2019-12-29,San Francisco,37.765946,-122.399044
8659,2019-12-28,San Francisco,37.765946,-122.399044
8635,2019-12-27,San Francisco,37.765946,-122.399044
8611,2019-12-26,San Francisco,37.765946,-122.399044
8587,2019-12-25,San Francisco,37.765946,-122.399044


In [91]:
df_sample3 = pd.merge(df_sample2, df_sample1,
                     how = 'right',
                     on = 'date') 
df_sample3.sort_values('date',ascending = False).head()

,date,county,lat,lon,sample
0,2019-12-29,San Francisco,37.765946,-122.399044,9.400000
1,2019-12-28,San Francisco,37.765946,-122.399044,9.208333
2,2019-12-27,San Francisco,37.765946,-122.399044,8.208333
3,2019-12-26,San Francisco,37.765946,-122.399044,7.043478
4,2019-12-25,San Francisco,37.765946,-122.399044,3.500000


# Same process for PM2.5 now for AQI
AQI = Air Quality Index
This is not done every day but every 1 - 3 days depending on how the EPA decided to track it.

In [92]:
email = 'candice.sessa10@gmail.com'
api_key = 'dunfox31'
state_code = '06'
county = '075'

response_daily = []
start = ["20190101", "20180101","20170101"]
end = ["20191231","20181231","20171231"]


for index in range(len(start)):
    # for each_state_code in ga_county_codes:
    url = f"https://aqs.epa.gov/data/api/dailyData/byCounty?email={email}&key={api_key}&param=88101&bdate={start[index]}&edate={end[index]}&state={state_code}&county={county}"
    response_daily.append(requests.get(url).json())

In [62]:
response_daily

[{'Header': [{'status': 'Success',
    'request_time': '2020-05-12T09:57:31-04:00',
    'url': 'https://aqs.epa.gov/data/api/dailyData/byCounty?email=candice.sessa10@gmail.com&key=dunfox31&param=88101&bdate=20190101&edate=20191231&state=06&county=075',
    'rows': 1798}],
  'Data': [{'state_code': '06',
    'county_code': '075',
    'site_number': '0005',
    'parameter_code': '88101',
    'poc': 3,
    'latitude': 37.765946,
    'longitude': -122.399044,
    'datum': 'WGS84',
    'parameter': 'PM2.5 - Local Conditions',
    'sample_duration': '24-HR BLK AVG',
    'pollutant_standard': 'PM25 Annual 2012',
    'date_local': '2019-12-31',
    'units_of_measure': 'Micrograms/cubic meter (LC)',
    'event_type': 'None',
    'observation_count': 1,
    'observation_percent': 100.0,
    'validity_indicator': 'Y',
    'arithmetic_mean': 9.9,
    'first_max_value': 9.9,
    'first_max_hour': 0,
    'aqi': 41,
    'method_code': '170',
    'method': 'Met One BAM-1020 Mass Monitor w/VSCC - Beta 

In [111]:
date = []
county = []
lat = []
lon = []
site = []
aqi = []


for x in range(0,3):
    for y in range(0,1772):
        date.append(response_daily[x]['Data'][y]['date_local'])
        aqi.append(response_daily[x]['Data'][y]['aqi'])

        
columns = ['date','aqi']
df_daily = pd.DataFrame(data = list(zip(date,aqi)), columns = columns).drop_duplicates()
df_daily.head()

,date,aqi
0,2019-01-01,NaN
1,2019-01-01,16.0
5,2019-01-02,NaN
6,2019-01-02,48.0
10,2019-01-03,NaN


In [112]:
# removing AQI NANs from dataset
df_daily1 = df_daily.dropna()
df_daily1.head()

,date,aqi
1,2019-01-01,16.0
6,2019-01-02,48.0
11,2019-01-03,71.0
16,2019-01-04,79.0
21,2019-01-05,25.0


# Merging AQI data with PM2.5 data

In [114]:
len(df_sample3)

1090

In [116]:
len(df_daily1)

1061

In [113]:
final = pd.merge(df_sample3, df_daily1,
              how = 'inner',
              on = 'date').re
final.head()

,date,county,lat,lon,sample,aqi
0,2019-12-26,San Francisco,37.765946,-122.399044,7.043478,29.0
1,2019-12-25,San Francisco,37.765946,-122.399044,3.500000,15.0
2,2019-12-24,San Francisco,37.765946,-122.399044,5.333333,22.0
3,2019-12-23,San Francisco,37.765946,-122.399044,5.416667,23.0
4,2019-12-22,San Francisco,37.765946,-122.399044,4.666667,19.0


In [117]:
len(final)

1060

# Exporting Data to CSV

In [119]:
final.to_csv('aqi_pm25.csv')